In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import List
from integration_agent.graph_builder import build_graph
from integration_agent.util.LLM import llm
from dotenv import load_dotenv

load_dotenv()

agent = None

async def call_agent(
    model: str,
    prompt: str,
    max_steps: int = 10,
    har_file_path: str = "turbo.har",
    cookie_path: str = "turbo.json",
    input_variables: dict = None,
):  
    
    llm.set_model(model)
    global agent
    graph, agent = build_graph(prompt, har_file_path, cookie_path)
    event_stream = graph.astream(
        {
            "master_node": None,
            "in_process_node": None,
            "to_be_processed_nodes": [],
            "in_process_node_dynamic_parts": [],
            "action_url": "",
            "input_variables": input_variables or {},  
        },
        {
            "recursion_limit": max_steps,
        },
    )
    async for event in event_stream:
        print("+++", event)

In [ ]:

model = "gpt-4o"
prompt = "Download my bank statement file."
input_variables = {
}
har_path = "dummy.har"
cookie_path = "dummy.json" 
max_steps = 10
await call_agent(model=model, prompt=prompt, har_file_path=har_path, cookie_path=cookie_path, max_steps=max_steps, input_variables=input_variables)

In [ ]:
from integration_agent.util.print import *

print_dag(agent.dag_manager.graph, agent.global_master_node_id)

In [ ]:
print_dag_in_reverse(agent.dag_manager.graph, to_generate_code=False)